# 3 Introducing Jupyter widgets
### Make your data analysis and visualisation interactive

<hr>
<a href="./02_Geospatial_Data_Acces.ipynb"><< 2 - Geospatial data access</a>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space> <a href="./04_Jupyter_Environment.ipynb">4 - Getting to know the Jupyter environment >></a>

## Jupyter widgets - an overview

Widgets can be used to build interactive GUIs for you notebooks. Get more information about widgets in the official [readthedocs](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#Selection-widgets).

### Some widget examples (before we get real)

In [1]:
# Load the widgets
import ipywidgets as widgets

#### Numeric widgets

In [8]:
widgets.IntSlider(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='vertical', #horizontal
    readout=True,
    readout_format='d'
)

IntSlider(value=7, continuous_update=False, description='Test:', max=10, orientation='vertical')

In [11]:
widgets.IntProgress(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Loading:',
    bar_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)

IntProgress(value=7, bar_style='warning', description='Loading:', max=10)

#### Boolean widgets

In [7]:
widgets.ToggleButton(
    value=False,
    description='Click me',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

ToggleButton(value=False, button_style='info', description='Click me', icon='check', tooltip='Description')

In [12]:
widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False
)

Checkbox(value=False, description='Check me')

#### Selection widgets

In [15]:
widgets.RadioButtons(
    options=['Jupyter is great', 'Jupyter is average', 'Do not understand the hype'],
#     value='pineapple',
    description='Survey:',
    disabled=False
)

RadioButtons(description='Survey:', options=('Jupyter is great', 'Jupyter is average', 'Do not understand the …

In [16]:
widgets.ToggleButtons(
    options=['Slow', 'Regular', 'Fast'],
    description='Speed:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

ToggleButtons(description='Speed:', options=('Slow', 'Regular', 'Fast'), tooltips=('Description of slow', 'Des…

#### String widgets

In [18]:
widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
)

Text(value='Hello World', description='String:', placeholder='Type something')

In [19]:
widgets.Textarea(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
)

Textarea(value='Hello World', description='String:', placeholder='Type something')

<hr>

## Example - Interactive application of climate graphs

Required libraries:
* [Ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/)
* [Plotly](https://plot.ly/) for interactive visualization [Registration required]
* [Widgets](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Basics.html)

In [1]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, FullScreenControl, Marker
import ipyleaflet

from IPython.display import display, clear_output
import ipywidgets as widgets
import numpy as np
import ee

import plotly.plotly as py
import plotly.graph_objs as go


In [2]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [3]:
ee.Initialize()

### Load ERA5 monthly ImageCollection 

In [4]:
era5_monthly = ee.ImageCollection('projects/ecmwf/era5_monthly')

In [5]:
era5_monthly.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'version': 1554369778704963,
 'id': 'projects/ecmwf/era5_monthly',
 'features': [{'type': 'Image',
   'bands': [{'id': 't2m',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'dimensions': [1440, 721],
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0.0, -180.125, 0.0, -0.25, 90.125]},
    {'id': 'tp',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'dimensions': [1440, 721],
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0.0, -180.125, 0.0, -0.25, 90.125]}],
   'version': 1554198981706109,
   'id': 'projects/ecmwf/era5_monthly/197901',
   'properties': {'system:time_start': 283996800000.0,
    'month': 1.0,
    'year': 1979.0,
    'system:footprint': {'type': 'LinearRing',
     'coordinates': [[-180.0, -90.0],
      [180.0, -90.0],
      [180.0, 90.0],
      [-180.0, 90.0],
      [-180.0, -90.0]]},
    'system:time_end': 286675200000.0,
    'system:asset_size': 8754505,
    'system:index': '

### Process mean precipitation for each month based on entire time series

In [6]:
era5_monthly_img = era5_monthly.limit(1).first()
collection_img_proj = era5_monthly_img.select(0).projection()

In [7]:
months = range(1,13)

# Store images in a list
img_list = []
for i in months:
    collection_filtered = era5_monthly.filter(ee.Filter.calendarRange(i,i, 'month'))
    collection_red = collection_filtered.reduce(ee.Reducer.mean())
    
    # if reducer function is applied to an image collection, the output does not have any projection information, as collection can contain \ 
    # images with different projection information. Thus, one can set the projection to each image
    collection_red_proj = collection_red.setDefaultProjection(collection_img_proj)
    img_list.append(collection_red_proj)
    
img_list[0].getInfo()

{'type': 'Image',
 'bands': [{'id': 't2m_mean',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0.0, -180.125, 0.0, -0.25, 90.125]},
  {'id': 'tp_mean',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0.0, -180.125, 0.0, -0.25, 90.125]}]}

### Build an Image Collection of the resulting image list

In [8]:
meanMonths_collection = ee.ImageCollection.fromImages(img_list)
meanMonths_collection.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 't2m_mean',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0.0, -180.125, 0.0, -0.25, 90.125]},
    {'id': 'tp_mean',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0.0, -180.125, 0.0, -0.25, 90.125]}],
   'properties': {'system:index': '0'}},
  {'type': 'Image',
   'bands': [{'id': 't2m_mean',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0.0, -180.125, 0.0, -0.25, 90.125]},
    {'id': 'tp_mean',
     'data_type': {'type': 'PixelType', 'precision': 'float'},
     'crs': 'EPSG:4326',
     'crs_transform': [0.25, 0.0, -180.125, 0.0, -0.25, 90.125]}],
   'properties': {'system:index': '1'}},
  {'type': 'Image',
   'bands': [{'id': 't2m_mean',
     'data_type': {'type': 'PixelType', 'preci

In [9]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [10]:
t2m_tmp = meanMonths_collection.select('t2m_mean').first()
tp_tmp = meanMonths_collection.select('tp_mean').first()

In [11]:
img_t2m = GetTileLayerUrl(t2m_tmp.visualize(min=250, max=330, palette=['#000080','#0000D9','#4000FF','#8000FF','#0080FF','#00FFFF','#00FF80','#80FF00','#DAFF00','#FFFF00','#FFF500','#FFDA00','#FFB000','#FFA400','#FF4F00','#FF2500','#FF0A00','#FF00FF']))
img_tp = GetTileLayerUrl(tp_tmp.visualize(min=0, max=1, palette=['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400','#FF0000']))

In [30]:
point = ee.Geometry.Point(lon,lat)
tp_point = meanMonths_collection.select('tp_mean').getRegion(point,500).getInfo()
t2m_point = meanMonths_collection.select('t2m_mean').getRegion(point,500).getInfo()

header_tp = tp_point[0]
data_tp = tp_point[1:]
ydata_tp = [row[4]*1000 for row in data_tp]
    
header_t2m = t2m_point[0]
data_t2m = t2m_point[1:]
ydata_t2m = [row[4]-273.2 for row in data_t2m]

max(ydata_t2m) + 2

35.525494384765636

In [32]:
def click(b):
    point = ee.Geometry.Point(lon,lat)
    tp_point = meanMonths_collection.select('tp_mean').getRegion(point,500).getInfo()
    t2m_point = meanMonths_collection.select('t2m_mean').getRegion(point,500).getInfo()
        
    header_tp = tp_point[0]
    data_tp = tp_point[1:]
    ydata_tp = [row[4]*1000 for row in data_tp]
    
    header_t2m = t2m_point[0]
    data_t2m = t2m_point[1:]
    ydata_t2m = [row[4]-273.2 for row in data_t2m]

    tp = go.Bar(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_tp,
        name='Total precipitation in mm',
        marker=dict(
            color='rgb(204,204,204)',
        ))
    
    t2m = go.Scatter(
        x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        y=ydata_t2m,
        name="2m air temperature in deg C",
        yaxis='y2')

    data = [tp,t2m]
    layout = go.Layout(
        title='Climate graph at location '+ str(round(lat,2)) + ' / '+ str(round(lon,2)) + ' (lat/lon)',
        yaxis=dict(
            title="Total precipitation in mm"
        ),
        yaxis2=dict(
            title="2 m air temperature in degC",
            overlaying='y',
            side='right',
            range=[0,max(ydata_t2m)+2]
        )
    )

    fig = go.Figure(data=data, layout=layout)
    with out:
        clear_output(wait=True)
        display(py.iplot(fig,filename='test'))


In [33]:
map1 = ipyleaflet.Map(
    zoom=2,
    layout={'height':'500px'},
)

map1.add_layer(ipyleaflet.TileLayer(url=img_t2m))
map1.add_layer(ipyleaflet.TileLayer(url=img_tp))
map1.add_control(ipyleaflet.LayersControl())

control = FullScreenControl()
map1.add_control(control)

map1

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [20]:
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        global lat, lon
        mark = ipyleaflet.Marker(location=kwargs.get('coordinates'))
        map1.add_layer(mark)
        location = mark.location
        lat, lon = location[0], location[1]        

map1.on_interaction(handle_click)

NameError: name 'map1' is not defined

In [35]:
out=widgets.Output()
button=widgets.Button(description='Plot climate graph')
button.on_click(click)
display(out)
display(button)

Output()

Button(description='Plot climate graph', style=ButtonStyle())

EEException: Earth Engine memory capacity exceeded.

<hr>
&copy; 2019 | Julia Wagemann
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>

In [ ]:
tp_point = meanMonths_collection.getRegion(point,500).getInfo()
point = ee.Geometry.Point(lon,lat)
header = tp_point[0]

data = tp_point[1:]
ydata = [row[4]*1000 for row in data]

tp = go.Bar(
    x=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
    y=ydata,
    name='Total precipitation in mm',
    marker=dict(
        color='rgb(204,204,204)',
    ))


data = [tp]
layout = go.Layout(
    title='Total precipitation in mm',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='test')



In [71]:
data_t2m

NameError: name 'data_t2m' is not defined